In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

C:\Users\Theo\anaconda3\envs\newenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Source Dataset**

In [2]:
classes = ["a09", "a13", "a14", "a15", "a07", "a03"]
persons = ["p1", "p2", "p3", "p4", "p5", "p6", "p7", "p8"]
x_train_source = []
y_train_source = []
for cl in range(6):
  for p in persons:
    for s in range(60):
      if s+1 < 10:
        link = "C:/Users/Theo/ECE685/project/data/"+classes[cl]+"/"+p+"/s0"+str(s+1)+".txt"
      else:
        link = "C:/Users/Theo/ECE685/project/data/"+classes[cl]+"/"+p+"/s"+str(s+1)+".txt"
      f = open(link, "r")
      data_point = []
      for x in f:
        ax = [float(i) for i in x.split(',')[:6]]
        data_point.append(ax)
      x_train_source.append(data_point)
      y_train_source.append(cl+1)

In [3]:
x_train_source = torch.from_numpy(np.array(x_train_source)).reshape(2880,125,6).type(torch.FloatTensor)
y_train_source = torch.from_numpy(np.array(y_train_source)).type(torch.FloatTensor)

**Target Dataset**

In [4]:
bax = torch.from_numpy(np.loadtxt("body_acc_x_train.txt")[:,:125]).reshape(7352,125,1).type(torch.FloatTensor)
bay = torch.from_numpy(np.loadtxt("body_acc_y_train.txt")[:,:125]).reshape(7352,125,1).type(torch.FloatTensor)
baz = torch.from_numpy(np.loadtxt("body_acc_z_train.txt")[:,:125]).reshape(7352,125,1).type(torch.FloatTensor)
bgx = torch.from_numpy(np.loadtxt("body_gyro_x_train.txt")[:,:125]).reshape(7352,125,1).type(torch.FloatTensor)
bgy = torch.from_numpy(np.loadtxt("body_gyro_y_train.txt")[:,:125]).reshape(7352,125,1).type(torch.FloatTensor)
bgz = torch.from_numpy(np.loadtxt("body_gyro_z_train.txt")[:,:125]).reshape(7352,125,1).type(torch.FloatTensor)
x_train = torch.cat((bax, bay, baz, bgx, bgy, bgz), 2)
y_train = torch.from_numpy(np.loadtxt("y_train.txt")).type(torch.FloatTensor) 

x_train = x_train[int(len(x_train)/2):]
y_train = y_train[int(len(y_train)/2):]

bax_test = torch.from_numpy(np.loadtxt("body_acc_x_test.txt")[:,:125]).reshape(2947,125,1).type(torch.FloatTensor)
bay_test = torch.from_numpy(np.loadtxt("body_acc_y_test.txt")[:,:125]).reshape(2947,125,1).type(torch.FloatTensor)
baz_test = torch.from_numpy(np.loadtxt("body_acc_z_test.txt")[:,:125]).reshape(2947,125,1).type(torch.FloatTensor)
bgx_test = torch.from_numpy(np.loadtxt("body_gyro_x_test.txt")[:,:125]).reshape(2947,125,1).type(torch.FloatTensor)
bgy_test = torch.from_numpy(np.loadtxt("body_gyro_y_test.txt")[:,:125]).reshape(2947,125,1).type(torch.FloatTensor)
bgz_test = torch.from_numpy(np.loadtxt("body_gyro_z_test.txt")[:,:125]).reshape(2947,125,1).type(torch.FloatTensor)
x_test = torch.cat((bax_test, bay_test, baz_test, bgx_test, bgy_test, bgz_test), 2)
y_test = torch.from_numpy(np.loadtxt("y_test.txt")).type(torch.FloatTensor) 

In [5]:
class LSTM(nn.Module):
  def __init__(self, n_input, n_hidden,
                 n_classes, drop_prob):
        super(LSTM, self).__init__()

        self.n_hidden = n_hidden
        self.n_classes = n_classes
        self.drop_prob = drop_prob
        self.n_input = n_input

        self.lstm1 = nn.LSTM(n_input, n_hidden, 1, dropout=self.drop_prob, batch_first=True)
        self.lstm2 = nn.LSTM(n_hidden, n_hidden, 1, dropout=self.drop_prob, batch_first=True)
        self.lstm3 = nn.LSTM(n_hidden, n_hidden, 1, dropout=self.drop_prob, batch_first=True)
        #self.batch = nn.BatchNorm2d()
        self.drop1 = nn.Dropout(drop_prob)
        self.fc1 = nn.Linear(n_hidden, 1)
        self.fc2 = nn.Linear(n_hidden, n_classes)
        self.activation = nn.Softmax(dim=1)

  def forward(self, x):
        out, (hn,cn) = self.lstm1(x)
        out, (hn,cn) = self.lstm2(out)
        out, (hn,cn) = self.lstm3(out)
        out = self.fc1(self.drop1(out))
        #out = self.fc2(self.drop1(out[:,:,0]))
        #out = self.fc1(out)
        out = self.fc2(out[:,:,0])
        out = self.activation(out)
        return out #(torch.argmax(out, dim = 1) + 1)

  def init_hidden(self, batch_size):
        ''' Initialize hidden state'''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        # if (train_on_gpu):
        if (torch.cuda.is_available() ):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                weight.new(self.n_layers, batch_size, self.n_hidden).zero_())

        return hidden


In [6]:
def accuracy(output, target):
  preds = torch.argmax(output, dim = 1)
  c = 0
  for i in range(len(preds)):
    if preds[i] == target[i]:
      c += 1
  return c/len(preds)

In [7]:
from sklearn.metrics import accuracy_score

def train(model, X, Y, num_epochs, num_classes, batch_size):
  criterion = nn.NLLLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=.001)
  #data = []
  #for i in range(len(X)):
  #  data.append([X[i], Y[i]])
  dataset = torch.utils.data.TensorDataset(torch.Tensor(np.array(X)), torch.Tensor(np.array(Y)))
  train_set_size = int(len(dataset)*0.8)
  val_set_size = len(dataset) - train_set_size
  train_set, val_set = torch.utils.data.random_split(dataset, [train_set_size, val_set_size])
  trainloader = torch.utils.data.DataLoader(train_set, shuffle=True, batch_size=100)
  valloader = torch.utils.data.DataLoader(val_set, shuffle=True, batch_size=100)

  for epoch in range(num_epochs):
    step_loss = 0
    train_acc = []
    model.train()
    for step, (x, y) in enumerate(trainloader):
      optimizer.zero_grad()
      outputs = model.forward(x.float())
      loss = criterion(torch.log(outputs), y.long()-1)
      train_acc.append(accuracy(outputs,y-1)*len(x))
      #loss.requires_grad = True
      loss.backward() #calculates the loss of the loss function
      optimizer.step()
      step_loss += loss
    print("Training Loss: ",step_loss.item())
    print("Training accuracy: ",np.sum(train_acc)/train_set_size)

    model.eval()
    val_loss = 0
    val_acc = []
    for step, (x, y) in enumerate(valloader):
      outputs = model.forward(x.float())
      loss = criterion(torch.log(outputs), y.long()-1)
      val_acc.append(accuracy(outputs,y-1)*len(x))
      #loss.requires_grad = True
      val_loss += loss
    print("Validation Loss: ",val_loss.item())
    print("Validation accuracy: ",np.sum(val_acc)/val_set_size)
  return model


In [8]:

#model = train(x_train_source, y_train_source, 20, 6, 100)
pre_trained_model = LSTM(6,125,6,.3)
pre_trained_model = train(pre_trained_model, x_train_source, y_train_source, 20, 6, 100)

C:\Users\Theo\anaconda3\envs\newenv\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Training Loss:  40.48585891723633
Training accuracy:  0.24869791666666666
Validation Loss:  8.416284561157227
Validation accuracy:  0.3541666666666667
Training Loss:  33.47208023071289
Training accuracy:  0.3263888888888889
Validation Loss:  7.6295695304870605
Validation accuracy:  0.3888888888888889
Training Loss:  31.206132888793945
Training accuracy:  0.3363715277777778
Validation Loss:  7.587491989135742
Validation accuracy:  0.3385416666666667
Training Loss:  30.2963924407959
Training accuracy:  0.3372395833333333
Validation Loss:  8.054112434387207
Validation accuracy:  0.3680555555555556
Training Loss:  30.48200035095215
Training accuracy:  0.3519965277777778
Validation Loss:  7.091183662414551
Validation accuracy:  0.4045138888888889
Training Loss:  28.955646514892578
Training accuracy:  0.40625
Validation Loss:  6.777156829833984
Validation accuracy:  0.5590277777777778
Training Loss:  28.13119888305664
Training accuracy:  0.4704861111111111
Validation Loss:  6.349762916564941

Spottune

In [9]:
class Agent(nn.Module):
  def __init__(self, n_input, n_hidden, n_output, drop_prob):
    super(Agent, self).__init__()

    self.n_input = n_input
    self.n_output = n_output
    self.n_hidden = n_hidden
    self.drop_prob = drop_prob

    self.lstm1 = nn.LSTM(n_input, n_hidden, 1, dropout=self.drop_prob, batch_first=True)
    self.lstm2 = nn.LSTM(n_hidden, n_hidden, 1, dropout=self.drop_prob, batch_first=True)
    #self.batch = nn.BatchNorm2d()
    self.drop1 = nn.Dropout(drop_prob)
    self.fc1 = nn.Linear(n_hidden, 1)
    self.fc2 = nn.Linear(n_hidden, n_output)
    self.activation = nn.Softmax(dim=1)
    
  def forward(self, x):
        out, (hn,cn) = self.lstm1(x)
        out, (hn, cn) = self.lstm2(out)
        out = self.fc1(self.drop1(out))
        #out = self.fc2(self.drop1(out[:,:,0]))
        #out = self.fc1(out)
        out = self.fc2(out[:,:,0])
        out = self.activation(out)
        return out #(torch.argmax(out, dim = 1) + 1)

  def init_hidden(self, batch_size):
        ''' Initialize hidden state'''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        # if (train_on_gpu):
        if (torch.cuda.is_available() ):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                weight.new(self.n_layers, batch_size, self.n_hidden).zero_())

        return hidden  

In [10]:
class SpottuneLSTM(nn.Module):
  def __init__(self, n_input, n_hidden,
                 n_classes, drop_prob):
        super(SpottuneLSTM, self).__init__()

        self.n_hidden = n_hidden
        self.n_classes = n_classes
        self.drop_prob = drop_prob
        self.n_input = n_input

        self.lstm1 = nn.LSTM(n_input, n_hidden, 1, dropout=self.drop_prob, batch_first=True)
        self.lstm2 = nn.LSTM(n_hidden, n_hidden, 1, dropout=self.drop_prob, batch_first=True)
        self.lstm3 = nn.LSTM(n_hidden, n_hidden, 1, dropout=self.drop_prob, batch_first=True)

        self.lstm1_frozen = nn.LSTM(n_input, n_hidden, 1, dropout=self.drop_prob, batch_first=True)
        self.lstm2_frozen = nn.LSTM(n_hidden, n_hidden, 1, dropout=self.drop_prob, batch_first=True)
        self.lstm3_frozen = nn.LSTM(n_hidden, n_hidden, 1, dropout=self.drop_prob, batch_first=True)


        #self.batch = nn.BatchNorm2d()
        self.drop1 = nn.Dropout(drop_prob)
        self.fc1 = nn.Linear(n_hidden, 1)
        self.fc2 = nn.Linear(n_hidden, n_classes)
        self.activation = nn.Softmax(dim=1)
  def lstm_output(self, lstm, lstm_frozen, policy, x, n_input):
    for i in range(len(policy)):
      ax = torch.reshape(x[i], (1, 125, n_input))
      if policy[i] == 0.:
        out_i, (_, _) = lstm_frozen(ax)
      else:
        out_i, (_, _) = lstm(ax)
      if i == 0:
        out = out_i
      else:
        out = torch.cat((out, out_i), 0)
    return out
  def forward(self, x, policy = None):
        out = self.lstm_output(self.lstm1, self.lstm1_frozen, policy[:, 0], x, self.n_input)
        #out, (hn,cn) = policy[:,0]*self.lstm1(x)+(1-policy[:,0])*self.lstm1_frozen(x)
        #out, (hn,cn) = policy[:,1]*self.lstm2(x)+(1-policy[:,1])*self.lstm2_frozen(x)
        out = self.lstm_output(self.lstm2, self.lstm2_frozen, policy[:, 1], out, self.n_hidden)
        #out, (hn,cn) = policy[:,2]*self.lstm3(x)+(1-policy[:,2])*self.lstm3_frozen(x)
        out = self.lstm_output(self.lstm3, self.lstm3_frozen, policy[:, 2], out, self.n_hidden)
        out = self.fc1(self.drop1(out))
        #out = self.fc2(self.drop1(out[:,:,0]))
        #out = self.fc1(out)
        out = self.fc2(out[:,:,0])
        out = self.activation(out)
        return out #(torch.argmax(out, dim = 1) + 1)

In [11]:
##It's working, it ust needed time and I wanted to go to sleep. Goodnight!!

spottunemodel = SpottuneLSTM(6,125,6,.3)
spottunemodel.lstm1_frozen.weight_ih_l0.requires_grad = False
spottunemodel.lstm1_frozen.weight_hh_l0.requires_grad = False
spottunemodel.lstm1_frozen.bias_ih_l0.requires_grad = False
spottunemodel.lstm1_frozen.bias_hh_l0.requires_grad = False

spottunemodel.lstm2_frozen.weight_ih_l0.requires_grad = False
spottunemodel.lstm2_frozen.weight_hh_l0.requires_grad = False
spottunemodel.lstm2_frozen.bias_ih_l0.requires_grad = False
spottunemodel.lstm2_frozen.bias_hh_l0.requires_grad = False

spottunemodel.lstm3_frozen.weight_ih_l0.requires_grad = False
spottunemodel.lstm3_frozen.weight_hh_l0.requires_grad = False
spottunemodel.lstm3_frozen.bias_ih_l0.requires_grad = False
spottunemodel.lstm3_frozen.bias_hh_l0.requires_grad = False
states = pre_trained_model.state_dict()
states["lstm1_frozen.weight_ih_l0"] = states["lstm1.weight_ih_l0"]
states["lstm1_frozen.weight_hh_l0"] = states["lstm1.weight_hh_l0"]
states["lstm1_frozen.bias_ih_l0"] = states["lstm1.bias_ih_l0"]
states["lstm1_frozen.bias_hh_l0"] = states["lstm1.bias_hh_l0"]
states["lstm2_frozen.weight_ih_l0"] = states["lstm2.weight_ih_l0"]
states["lstm2_frozen.weight_hh_l0"] = states["lstm2.weight_hh_l0"]
states["lstm2_frozen.bias_ih_l0"] = states["lstm2.bias_ih_l0"]
states["lstm2_frozen.bias_hh_l0"] = states["lstm2.bias_hh_l0"]
states["lstm3_frozen.weight_ih_l0"] = states["lstm3.weight_ih_l0"]
states["lstm3_frozen.weight_hh_l0"] = states["lstm3.weight_hh_l0"]
states["lstm3_frozen.bias_ih_l0"] = states["lstm3.bias_ih_l0"]
states["lstm3_frozen.bias_hh_l0"] = states["lstm3.bias_hh_l0"]
spottunemodel.load_state_dict(states)

def spottunetrain(model, agent, X, Y, num_epochs, num_classes, batch_size):
  criterion = nn.NLLLoss()
  optimizer_model = torch.optim.Adam(model.parameters(), lr=.001)
  optimizer_agent = torch.optim.Adam(agent.parameters(), lr=.001)
  dataset = torch.utils.data.TensorDataset(torch.Tensor(np.array(X)), torch.Tensor(np.array(Y)))
  train_set_size = int(len(dataset)*0.8)
  val_set_size = len(dataset) - train_set_size
  train_set, val_set = torch.utils.data.random_split(dataset, [train_set_size, val_set_size])
  trainloader = torch.utils.data.DataLoader(train_set, shuffle=True, batch_size=100)
  valloader = torch.utils.data.DataLoader(val_set, shuffle=True, batch_size=100)

  for epoch in range(num_epochs):
    model.train()
    step_loss = 0
    train_acc = []
    for step, (x, y) in enumerate(trainloader):
      optimizer_model.zero_grad()
      optimizer_agent.zero_grad()
      policy = agent(x)
      action = F.gumbel_softmax(policy, tau=1, hard=True)
      outputs = model.forward(x.float(), action)
      loss = criterion(torch.log(outputs), y.long()-1)
      train_acc.append(accuracy(outputs,y-1)*len(x))
      #loss.requires_grad = True
      loss.backward() #calculates the loss of the loss function
      if action[0][0] == 0:
        if spottunemodel.lstm1.weight_ih_l0.grad is not None:
          spottunemodel.lstm1.weight_ih_l0.grad.data.zero_()
        if spottunemodel.lstm1.weight_hh_l0.grad is not None:
          spottunemodel.lstm1.weight_hh_l0.grad.data.zero_()
        if spottunemodel.lstm1.bias_ih_l0.grad is not None:
          spottunemodel.lstm1.bias_ih_l0.grad.data.zero_()
        if spottunemodel.lstm1.bias_hh_l0.grad is not None:
          spottunemodel.lstm1.bias_hh_l0.grad.data.zero_()
      if action[0][1] == 0:
        if spottunemodel.lstm2.weight_ih_l0.grad is not None:
          spottunemodel.lstm2.weight_ih_l0.grad.data.zero_()
        if spottunemodel.lstm2.weight_hh_l0.grad is not None:
          spottunemodel.lstm2.weight_hh_l0.grad.data.zero_()
        if spottunemodel.lstm2.bias_ih_l0.grad is not None:
          spottunemodel.lstm2.bias_ih_l0.grad.data.zero_()
        if spottunemodel.lstm2.bias_hh_l0.grad is not None:
          spottunemodel.lstm2.bias_hh_l0.grad.data.zero_()
      if action[0][2] == 0:
        if spottunemodel.lstm3.weight_ih_l0.grad is not None:
          spottunemodel.lstm3.weight_ih_l0.grad.data.zero_()
        if spottunemodel.lstm3.weight_hh_l0.grad is not None:
          spottunemodel.lstm3.weight_hh_l0.grad.data.zero_()
        if spottunemodel.lstm3.bias_ih_l0.grad is not None:
          spottunemodel.lstm3.bias_ih_l0.grad.data.zero_()
        if spottunemodel.lstm3.bias_hh_l0.grad is not None:
          spottunemodel.lstm3.bias_hh_l0.grad.data.zero_()
      optimizer_model.step()
      optimizer_agent.step()
      step_loss += loss
    print("Training Loss: ",step_loss.item())
    print("Training Accuracy: ",np.sum(train_acc)/train_set_size)

    model.eval()
    val_loss = 0
    val_acc = []
    for step, (x, y) in enumerate(valloader):
      policy = agent(x)
      action = F.gumbel_softmax(policy, tau=1, hard=True)
      outputs = model.forward(x.float(), action)
      loss = criterion(torch.log(outputs), y.long()-1)
      val_acc.append(accuracy(outputs,y-1)*len(x))
      val_loss += loss
    print("Validation Loss: ",val_loss.item())
    print("Validation Accuracy: ",np.sum(val_acc)/val_set_size)
  return model

agent = Agent(6, 125, 3, 0.3)
#spottunemodel = spottunetrain(spottunemodel, agent, x_train, y_train, 20, 6, 100)
spottunemodel = spottunetrain(spottunemodel, agent, x_train, y_train, 10, 6, 1)

Training Loss:  147.43612670898438
Training Accuracy:  0.1683673469387755
Validation Loss:  21.207746505737305
Validation Accuracy:  0.1644021739130435
Training Loss:  60.96635437011719
Training Accuracy:  0.21768707482993196
Validation Loss:  13.89486312866211
Validation Accuracy:  0.26358695652173914
Training Loss:  50.20680618286133
Training Accuracy:  0.2578231292517007
Validation Loss:  12.968700408935547
Validation Accuracy:  0.24048913043478262
Training Loss:  45.84107971191406
Training Accuracy:  0.295578231292517
Validation Loss:  11.377649307250977
Validation Accuracy:  0.3138586956521739
Training Loss:  42.45390319824219
Training Accuracy:  0.3217687074829932
Validation Loss:  11.117589950561523
Validation Accuracy:  0.31657608695652173
Training Loss:  41.06414794921875
Training Accuracy:  0.33163265306122447
Validation Loss:  10.664321899414062
Validation Accuracy:  0.327445652173913
Training Loss:  39.267181396484375
Training Accuracy:  0.33945578231292517
Validation Loss:

In [12]:
testset = torch.utils.data.TensorDataset(torch.Tensor(np.array(x_test)), torch.Tensor(np.array(y_test)))
testloader = torch.utils.data.DataLoader(testset, shuffle=True, batch_size=len(testset))
criterion = nn.NLLLoss()
spottunemodel.eval()
test_loss = 0
test_acc = []
for step, (x, y) in enumerate(testloader):
  policy = agent(x)
  action = F.gumbel_softmax(policy, tau=1, hard=True)
  outputs = spottunemodel.forward(x.float(), action)
  loss = criterion(torch.log(outputs), y.long()-1)
  test_acc.append(accuracy(outputs,y-1)*len(x))
  test_loss += loss
print("Test Loss: ",test_loss.item())
print("Test Accuracy: ",np.sum(test_acc)/len(x_test))

Test Loss:  1.4175454378128052
Test Accuracy:  0.31964709874448594


**Benchmark**

In [13]:
benchmark_model = LSTM(6,125,6,.3)
benchmark_model = train(benchmark_model, x_train, y_train, 10, 6, 100)

Training Loss:  53.328147888183594
Training accuracy:  0.19931972789115646
Validation Loss:  13.563400268554688
Validation accuracy:  0.3029891304347826
Training Loss:  39.78975296020508
Training accuracy:  0.33095238095238094
Validation Loss:  9.353944778442383
Validation accuracy:  0.32608695652173914
Training Loss:  34.06383514404297
Training accuracy:  0.3608843537414966
Validation Loss:  9.096637725830078
Validation accuracy:  0.33695652173913043
Training Loss:  33.830284118652344
Training accuracy:  0.34183673469387754
Validation Loss:  8.9976806640625
Validation accuracy:  0.3342391304347826
Training Loss:  33.53144073486328
Training accuracy:  0.34965986394557824
Validation Loss:  9.065139770507812
Validation accuracy:  0.33559782608695654
Training Loss:  33.273189544677734
Training accuracy:  0.36156462585034016
Validation Loss:  8.917169570922852
Validation accuracy:  0.35054347826086957
Training Loss:  33.17144012451172
Training accuracy:  0.35714285714285715
Validation Loss

In [14]:
testset = torch.utils.data.TensorDataset(torch.Tensor(np.array(x_test)), torch.Tensor(np.array(y_test)))
testloader = torch.utils.data.DataLoader(testset, shuffle=True, batch_size=len(testset))
criterion = nn.NLLLoss()
benchmark_model.eval()
test_loss = 0
test_acc = []
for step, (x, y) in enumerate(testloader):
  outputs = benchmark_model.forward(x.float())
  loss = criterion(torch.log(outputs), y.long()-1)
  test_acc.append(accuracy(outputs,y-1)*len(x))
  test_loss += loss
print("Test Loss: ",test_loss.item())
print("Test Accuracy: ",np.sum(test_acc)/len(x_test))

Test Loss:  1.146884560585022
Test Accuracy:  0.34781133355955207
